In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# pip install numpy==2.0 pandas tsfresh

In [3]:
import numpy as np
import csv
import pandas as pd
import glob
import os
import re

In [4]:
# pip install tsfresh
import tsfresh

In [5]:
# os.environ["MallocStackLogging"] = "0"

In [6]:
# main_folder = os.path.join(os.getcwd(), 'q1q17dr25_mcmcChains')                 # Should be changed with your actual folder name
main_folder = os.path.join(os.getcwd(), 'main_folders')
koi_table = 'q1_q17_dr25_sup_koi.csv'                                           # Should be changed with your actual file name
startKOI = 1101                                                                      # Should be changed
endKOI = 1126#1126                                                                    # Should be changed
koi_names = [f"koi{i}.n" for i in range(startKOI, endKOI + 1)]

tremove_files = []

for subfolder in koi_names:
    search_pattern = os.path.join(main_folder, subfolder, 'tremove.*.dat')
    tremove_files.extend(glob.glob(search_pattern))
    # tremove_files.append(glob.glob(search_pattern))

print(tremove_files)

['/Users/wongweixiang/Desktop/Github/NTHU/Machine_Learning/Final/main_folders/koi1101.n/tremove.1.dat', '/Users/wongweixiang/Desktop/Github/NTHU/Machine_Learning/Final/main_folders/koi1101.n/tremove.2.dat', '/Users/wongweixiang/Desktop/Github/NTHU/Machine_Learning/Final/main_folders/koi1102.n/tremove.4.dat', '/Users/wongweixiang/Desktop/Github/NTHU/Machine_Learning/Final/main_folders/koi1102.n/tremove.1.dat', '/Users/wongweixiang/Desktop/Github/NTHU/Machine_Learning/Final/main_folders/koi1102.n/tremove.2.dat', '/Users/wongweixiang/Desktop/Github/NTHU/Machine_Learning/Final/main_folders/koi1102.n/tremove.3.dat', '/Users/wongweixiang/Desktop/Github/NTHU/Machine_Learning/Final/main_folders/koi1103.n/tremove.1.dat', '/Users/wongweixiang/Desktop/Github/NTHU/Machine_Learning/Final/main_folders/koi1104.n/tremove.1.dat', '/Users/wongweixiang/Desktop/Github/NTHU/Machine_Learning/Final/main_folders/koi1105.n/tremove.1.dat', '/Users/wongweixiang/Desktop/Github/NTHU/Machine_Learning/Final/main_fol

In [7]:
print(len(tremove_files))
# for file in tremove_files:
#     print(file)

38


In [8]:
def extract_file_number(filename):
    match = re.search(r'tremove\.(\d+)\.dat', filename)
    return int(match.group(1)) if match else None

def extract_folder_number(folder_name):
    match = re.search(r'koi(\d+)\.n', folder_name)
    return int(match.group(1)) if match else None

In [9]:
for file in tremove_files:
    df_list = []
    
    folder_name = os.path.basename(os.path.dirname(file))
    folder_num = extract_folder_number(folder_name)
    file_num = extract_file_number(file)
    id_name = f"K{folder_num:05}.{file_num:02}"

    df = pd.read_csv(file, delimiter='\s+', header=None)
    df.insert(0, 'id', id_name)
    df = df.rename(columns={0: 'time', 1: 'light_curve', 2: 'error'})
    print("ID : ",id_name)
    features = tsfresh.extract_features(df, column_id='id', column_sort='time', column_value='light_curve')
    # df_list.append(features)
    # print(features)

    # tremove_features = pd.concat(df_list, ignore_index=False)
    
    koi_df = pd.read_csv(koi_table)
    koi_df.drop(columns='loc_rowid')
    koi_df.index = koi_df['kepoi_name']

    # filtered_koi_df = koi_df[(koi_df['kepoi_name'] >= f'K00{startKOI}.*') & (koi_df['kepoi_name'] <= f'K00{endKOI+1}.*')]
    # candidate = filtered_koi_df[filtered_koi_df['koi_disposition'] == "CANDIDATE"]
    # confirmed = filtered_koi_df[filtered_koi_df['koi_disposition'] == "CONFIRMED"]
    # false_positive = filtered_koi_df[filtered_koi_df['koi_disposition'] == "FALSE POSITIVE"]
    # print("CANDIDATE : ",len(candidate),", CONFIRMED : ",len(confirmed),", FALSE POSITIVE : ",len(false_positive))
    # filtered_koi_df

    features = pd.merge(features, koi_df['koi_disposition'], left_index=True, right_index=True)

    features['label'] = np.where(features['koi_disposition'] == 'CONFIRMED',1,0)
    # features['label'] = np.where(features['koi_disposition'] == 'CONFIRMED',2,"X")
    # for label in features['label']:
    #     if label == "X":
    #         features['label'] = np.where(features['koi_disposition'] == 'FALSE POSITIVE',0,1)
    features = features.drop(columns=['koi_disposition'])
    # candidate = features[features['label'] == "CANDIDATE"]
    confirmed = features[features['label'] == 1 ]
    false_positive = features[features['label'] == 0 ]
    print("CONFIRMED : ",len(confirmed),", FALSE POSITIVE : ",len(false_positive))
    features.to_csv(f'koi_features_{id_name}.csv', index=True)

Feature Extraction: 100%|██████████| 1/1 [00:03<00:00,  3.91s/it]

CONFIRMED :  1 , FALSE POSITIVE :  0



Feature Extraction: 100%|██████████| 1/1 [00:03<00:00,  3.89s/it]

CONFIRMED :  0 , FALSE POSITIVE :  1



Feature Extraction: 100%|██████████| 1/1 [1:02:23<00:00, 3743.41s/it]


CONFIRMED :  1 , FALSE POSITIVE :  0


python(3194) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(3195) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(3196) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(3197) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Feature Extraction: 100%|██████████| 1/1 [1:37:16<00:00, 5836.64s/it]


CONFIRMED :  1 , FALSE POSITIVE :  0


python(4461) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(4462) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(4463) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(4464) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Feature Extraction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# tremove_features